In [1]:
import os

os.chdir('../..')

In [2]:
from os.path import join

import numpy as np
import pandas as pd

import modules.point_processing as pp

In [3]:
df_truth = pd.read_pickle('results/dataframes/df_truth.pkl')

In [4]:
trial_names = df_truth.index.get_level_values(0).unique().values

In [5]:
dict_hypo, dict_selected = {}, {}

for trial_name in trial_names:
    
    dict_hypo[trial_name] = pd.read_pickle('data/kinect/processed/hypothesis/' + trial_name + '.pkl')
    dict_selected[trial_name] = pd.read_pickle('data/kinect/best_pos/' + trial_name + '.pkl')

df_hypo = pd.concat(dict_hypo)
df_selected = pd.concat(dict_selected)


In [6]:
foot_parts = ['L_FOOT', 'R_FOOT']

In [7]:
# Truth positions on frames with head and both feet
df_truth = df_truth.dropna(subset=['HEAD'] + foot_parts)

In [8]:
index_intersect = df_truth.index.intersection(df_selected.index)

In [9]:
df_truth = df_truth.loc[index_intersect]
df_selected = df_selected.loc[index_intersect]

In [10]:
frames = df_truth.index

In [11]:
# df_hypo_matched = df_hypo.loc[frames]
# df_selected_matched = df_selected.reindex(frames).dropna()

In [12]:
df_truth_modified = pd.DataFrame(index=frames, 
                                 columns=foot_parts)

for frame in frames:
    
    foot_proposals = df_hypo.loc[frame, 'FOOT']
    true_foot_l, true_foot_r = df_truth.loc[frame, foot_parts]
        
    proposal_closest_l, _ = pp.closest_point(foot_proposals, true_foot_l.reshape(1, -1))
    proposal_closest_r, _ = pp.closest_point(foot_proposals, true_foot_r.reshape(1, -1))
    
    df_truth_modified.loc[frame, foot_parts[0]] = proposal_closest_l
    df_truth_modified.loc[frame, foot_parts[1]] = proposal_closest_r

In [13]:
pp.position_accuracy(np.stack(df_truth_modified.L_FOOT), np.stack(df_truth.L_FOOT))

0.8347676419965576

In [14]:
pp.position_accuracy(np.stack(df_truth_modified.R_FOOT), np.stack(df_truth.R_FOOT))

0.8261617900172117

In [15]:
df_selected_assigned = pd.DataFrame(index=frames,
                                    columns=foot_parts)

for frame in frames:
    
    points_selected = np.stack(df_selected.loc[frame][foot_parts])
    points_truth = np.stack(df_truth_modified.loc[frame])
    
    assigned_l, assigned_r = pp.correspond_points(points_truth, points_selected)
    
    df_selected_assigned.loc[frame, foot_parts[0]] = assigned_l
    df_selected_assigned.loc[frame, foot_parts[1]] = assigned_r

In [16]:
pp.position_accuracy(np.stack(df_selected_assigned.L_FOOT), np.stack(df_truth_modified.L_FOOT))

0.9913941480206541

In [17]:
pp.position_accuracy(np.stack(df_selected_assigned.R_FOOT), np.stack(df_truth_modified.R_FOOT))

0.9827882960413081